In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
pos = pd.read_json('result/pos_embeddings.json')
neg = pd.read_json('result/neg_embeddings.json')
X_pos = np.array(pos.embedding.map(np.array).to_list())
X_neg = np.array(neg.embedding.map(np.array).to_list())

In [4]:
len(pos)

3352

In [5]:
X_pos = np.load("result/pos_image_embeddings.npz")['arr_0']

In [6]:
len(X_pos)

2934

In [16]:
# 处理下采样负样本
from sklearn.utils import resample

def balance_data_down_sampling(X_pos, X_neg):
    """
    均衡正类和负类的数据数量通过下采样少数类。

    Parameters:
    X_pos (np.array): 正类特征数据
    y_pos (np.array): 正类标签数据
    X_neg (np.array): 负类特征数据
    y_neg (np.array): 负类标签数据

    Returns:
    np.array, np.array: 均衡后的特征数据和标签数据的index
    """
    # 确定每类应有的样本数量
    n_samples = min(X_pos.shape[0], X_neg.shape[0])

    # 下采样正类
    X_pos_downsampled = resample(X_pos, n_samples=n_samples, random_state=42)
    # 下采样负类
    X_neg_downsampled = resample(X_neg, n_samples=n_samples, random_state=42)

    print(X_pos_downsampled.shape, X_neg_downsampled.shape)
    
    # 合并下采样后的正类和负类数据
    X_balanced = np.vstack((X_pos_downsampled, X_neg_downsampled))
    y_pos_downsampled = np.ones(X_pos_downsampled.shape[0])
    y_neg_downsampled = np.zeros(X_neg_downsampled.shape[0])
    y_balanced = np.hstack((y_pos_downsampled, y_neg_downsampled))

    return X_balanced, y_balanced



X_pos = np.load("../../data/features_0507/pos/data_pos.npz")['arr_0']
X_pos_2 = np.load("result/pos_image_embeddings.npz")['arr_0']
X_neg = np.load("../../data/features_0507/neg_image_embeddings.npz")['arr_0']
X_neg_2 = np.load("result/neg_image_embeddings.npz")['arr_0']
X_pos = np.concatenate((X_pos, X_pos_2), axis=0)
X_neg = np.concatenate((X_neg, X_neg_2), axis=0)

del X_neg_2
del X_pos_2
X_balanced, y_balanced = balance_data_down_sampling(X_pos, X_neg)
# 保存下采样后的数据
np.savez_compressed("result/balanced_data.npz", X_balanced, y_balanced)

(251130, 768) (251130, 768)


In [19]:
X_pos = np.load("../../data/features_0507/pos/data_pos.npz")['arr_0']
# X_pos_2 = np.load("result/pos_image_embeddings.npz")['arr_0']
# X_neg = np.load("../../data/features_0507/neg_image_embeddings.npz")['arr_0']
# X_neg_2 = np.load("result/neg_image_embeddings.npz")['arr_0']
# X_pos = np.concatenate((X_pos, X_pos_2), axis=0)
# X_neg = np.concatenate((X_neg, X_neg_2), axis=0)
# y_pos = np.ones(X_pos.shape[0], dtype=int)
# y_neg = np.zeros(X_neg.shape[0], dtype=int)
np.save("../../data/features_0507/pos/data_pos.npy", X_pos)


In [4]:
# print(len(X_pos))
print(len(X_neg))

# 将X_neg分成6份，每份100w数据，存在../../data/features_0507/中, 保存为npy文件
for i in range(60000):
    np.save("../../data/features_0507/neg/data_neg_{}".format(i), X_neg[i*100:(i+1)*100])

6374817


In [45]:
X_pos_n = X_pos / (np.linalg.norm(X_pos, axis=-1, keepdims=True) + 1e-6)
X_neg_n = X_neg / (np.linalg.norm(X_neg, axis=-1, keepdims=True) + 1e-6)

In [5]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def pca_whitening(X):
    """
    对输入数据X执行PCA白化处理。
    
    参数:
    X (np.array): 输入数据，形状为 (n_samples, n_features)
    
    返回:
    np.array: 白化后的数据
    """
    # 首先对数据进行标准化（去均值并单位方差）
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # 执行PCA
    pca = PCA(whiten=True)  # 设置whiten=True来进行白化
    X_whitened = pca.fit_transform(X_scaled)
    
    return X_whitened

In [3]:
X_pos_train, X_pos_test, y_pos_train, y_pos_test = train_test_split(X_pos, np.ones(X_pos.shape[0], dtype=int), test_size=0.33, random_state=42)
X_neg_train, X_neg_test, y_neg_train, y_neg_test = train_test_split(X_neg, np.zeros(X_neg.shape[0], dtype=int), test_size=0.33, random_state=42)

In [3]:
class Embed_data:
    def __init__(self, X, y, scale_data=False):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            if scale_data:
                X = StandardScaler().fit_transform(X)
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [2]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim
        self.fc0 = nn.Linear(input_dim, hidden_dims[0])
        self.activate0 = nn.GELU()
        for i in range(len(hidden_dims)-1):
            setattr(self, f'fc{i+1}', nn.Linear(hidden_dims[i], hidden_dims[i+1]))
            setattr(self, f'activate{i+1}', nn.GELU())
        self.fcout = nn.Linear(hidden_dims[-1], output_dim)
        self.final_activate = nn.Sigmoid()

    def forward(self, x):
        x = self.fc0(x)
        x = self.activate0(x)
        for i in range(len(self.hidden_dims)-1):
            x = getattr(self, f'fc{i+1}')(x)
            x = getattr(self, f'activate{i+1}')(x)
        x = self.fcout(x)
        # x = self.final_activate(x)
        return x

In [4]:
from sklearn.utils import resample

def balance_data_by_number(pos_number, neg_number):
    """
    均衡正类和负类的数据数量通过上采样少数类。

    Parameters:
    pos_number(int):   正类样本的数目
    neg_number(int):   负类样本的数目
    Returns:
    np.array, np.array: 均衡后的特征数据和标签数据的index
    """
    X_pos_index = np.arange(pos_number)
    X_neg_index = np.arange(neg_number)
    n_samples = max(pos_number, neg_number)
    
    X_pos_index_upsampled = resample(X_pos_index, n_samples=n_samples, random_state=42)
    X_neg_index_upsampled = resample(X_neg_index, n_samples=n_samples, random_state=42)

    print(f"pos = {X_pos_index_upsampled.shape}, neg = {X_neg_index_upsampled.shape}")

    return np.arange(n_samples), X_pos_index_upsampled, X_neg_index_upsampled

In [5]:
from sklearn.utils import resample

def balance_data(X_pos, X_neg):
    """
    均衡正类和负类的数据数量通过上采样少数类。

    Parameters:
    X_pos (np.array): 正类特征数据
    y_pos (np.array): 正类标签数据
    X_neg (np.array): 负类特征数据
    y_neg (np.array): 负类标签数据

    Returns:
    np.array, np.array: 均衡后的特征数据和标签数据的index
    """
    # 确定每类应有的样本数量
    n_samples = max(X_pos.shape[0], X_neg.shape[0])
    X_all = np.vstack((X_pos, X_neg))
    y_all = np.hstack((np.ones(X_pos.shape[0]), np.zeros(X_neg.shape[0])))
    X_pos_index = np.arange(X_pos.shape[0])
    X_neg_index = np.arange(X_pos.shape[0], X_pos.shape[0]+X_neg.shape[0])

    # 上采样正类
    X_pos_upsampled = resample(X_pos_index, n_samples=n_samples, random_state=42)
    # 上采样负类（如果需要）
    X_neg_upsampled = resample(X_neg_index, n_samples=n_samples, random_state=42)

    print(X_pos_upsampled.shape, X_neg_upsampled.shape)
    
    # 合并上采样后的正类和负类数据
    X_balanced_index = np.vstack((X_pos_upsampled, X_neg_upsampled))
    # y_balanced = np.hstack((y_pos_upsampled, y_neg_upsampled))

    return X_balanced_index, X_all, y_all



In [6]:
import math

class CosineAnnealingLR(torch.optim.lr_scheduler._LRScheduler):
    def __init__(
        self,
        optimizer: torch.optim.Optimizer,
        warmup_epochs: int,
        max_epochs: int,
        warmup_start_lr: float = 0.00001,
        eta_min: float = 0.00001,
        last_epoch: int = -1,
    ):
        """
        Args:
            optimizer (torch.optim.Optimizer):
                最適化手法インスタンス
            warmup_epochs (int):
                linear warmupを行うepoch数
            max_epochs (int):
                cosine曲線の終了に用いる 学習のepoch数
            warmup_start_lr (float):
                linear warmup 0 epoch目の学習率
            eta_min (float):
                cosine曲線の下限
            last_epoch (int):
                cosine曲線の位相オフセット
        学習率をmax_epochsに至るまでコサイン曲線に沿ってスケジュールする
        epoch 0からwarmup_epochsまでの学習曲線は線形warmupがかかる
        https://pytorch-lightning-bolts.readthedocs.io/en/stable/schedulers/warmup_cosine_annealing.html
        """
        self.warmup_epochs = warmup_epochs
        self.max_epochs = max_epochs
        self.warmup_start_lr = warmup_start_lr
        self.eta_min = eta_min
        super().__init__(optimizer, last_epoch)
        return None

    def get_lr(self):
        if self.last_epoch == 0:
            return [self.warmup_start_lr] * len(self.base_lrs)
        if self.last_epoch < self.warmup_epochs:
            return [
                group["lr"] + (base_lr - self.warmup_start_lr) / (self.warmup_epochs - 1)
                for base_lr, group in zip(self.base_lrs, self.optimizer.param_groups)
            ]
        if self.last_epoch == self.warmup_epochs:
            return self.base_lrs
        if (self.last_epoch - 1 - self.max_epochs) % (2 * (self.max_epochs - self.warmup_epochs)) == 0:
            return [
                group["lr"] + (base_lr - self.eta_min) * (1 - math.cos(math.pi / (self.max_epochs - self.warmup_epochs))) / 2
                for base_lr, group in zip(self.base_lrs, self.optimizer.param_groups)
            ]

        return [
            (1 + math.cos(math.pi * (self.last_epoch - self.warmup_epochs) / (self.max_epochs - self.warmup_epochs)))
            / (1 + math.cos(math.pi * (self.last_epoch - self.warmup_epochs - 1) / (self.max_epochs - self.warmup_epochs)))
            * (group["lr"] - self.eta_min)
            + self.eta_min
            for group in self.optimizer.param_groups
        ]

In [14]:
from torch.utils.data.dataloader import default_collate
from typing import List, Tuple
from torch import Tensor

class RandomMixUp(torch.nn.Module):
    """Randomly apply MixUp to the provided batch and targets.
    The class implements the data augmentations as described in the paper
    `"mixup: Beyond Empirical Risk Minimization" <https://arxiv.org/abs/1710.09412>`_.

    Args:
        num_classes (int): number of classes used for one-hot encoding.
        p (float): probability of the batch being transformed. Default value is 0.5.
        alpha (float): hyperparameter of the Beta distribution used for mixup.
            Default value is 1.0.
        inplace (bool): boolean to make this transform inplace. Default set to False.
    """

    def __init__(self, num_classes: int, p: float = 0.5, alpha: float = 1.0, inplace: bool = False) -> None:
        super().__init__()

        if num_classes < 1:
            raise ValueError(
                f"Please provide a valid positive value for the num_classes. Got num_classes={num_classes}"
            )

        if alpha <= 0:
            raise ValueError("Alpha param can't be zero.")

        self.num_classes = num_classes
        self.p = p
        self.alpha = alpha
        self.inplace = inplace

    def forward(self, batch: Tensor, target: Tensor) -> Tuple[Tensor, Tensor]:
        """
        Args:
            batch (Tensor): Float tensor of size (B, C)
            target (Tensor): Integer tensor of size (B, )

        Returns:
            Tensor: Randomly transformed batch.
        """
        if batch.ndim != 2:
            raise ValueError(f"Batch ndim should be 2. Got {batch.ndim}")
        if target.ndim != 1:
            raise ValueError(f"Target ndim should be 1. Got {target.ndim}")
        if not batch.is_floating_point():
            raise TypeError(f"Batch dtype should be a float tensor. Got {batch.dtype}.")
        if target.dtype != torch.int64:
            raise TypeError(f"Target dtype should be torch.int64. Got {target.dtype}")

        if not self.inplace:
            batch = batch.clone()
            target = target.clone()

        if target.ndim == 1:
            target = torch.nn.functional.one_hot(target, num_classes=self.num_classes).to(dtype=batch.dtype)

        if torch.rand(1).item() >= self.p:
            return batch, target

        # It's faster to roll the batch by one instead of shuffling it to create image pairs
        batch_rolled = batch.roll(1, 0)
        target_rolled = target.roll(1, 0)

        # Implemented as on mixup paper, page 3.
        lambda_param = float(torch._sample_dirichlet(torch.tensor([self.alpha, self.alpha]))[0])
        batch_rolled.mul_(1.0 - lambda_param)
        batch.mul_(lambda_param).add_(batch_rolled)

        target_rolled.mul_(1.0 - lambda_param)
        target.mul_(lambda_param).add_(target_rolled)

        # print(target.dtype)

        return batch, target

    def __repr__(self) -> str:
        s = (
            f"{self.__class__.__name__}("
            f"num_classes={self.num_classes}"
            f", p={self.p}"
            f", alpha={self.alpha}"
            f", inplace={self.inplace}"
            f")"
        )
        return s

In [7]:
from torch.utils.data.sampler import Sampler
class CustomBatchSampler(Sampler):
    def __init__(self, pos_data_source, neg_data_source, pos_data_index, neg_data_index, total_index, batch_size, drop_last):
        self.pos_data_source = pos_data_source
        self.neg_data_source = neg_data_source
        self.pos_data_index = pos_data_index
        self.neg_data_index = neg_data_index
        self.batch_size = batch_size
        self.drop_last = drop_last
        self.total_index = total_index
        print(f"{len(total_index)}, {len(pos_data_index)}, {len(neg_data_index)}, {batch_size%2}")
        assert (len(total_index) == len(pos_data_index)) and (len(total_index) == len(neg_data_index)) and (batch_size%2 == 0)

    def __iter__(self):
        batch = []
        for idx in self.total_index:
            batch.append(self.pos_data_source[self.pos_data_index[idx]])
            batch.append(self.neg_data_source[self.neg_data_index[idx]])
            if len(batch) == self.batch_size:
                # 随机打乱batch
                np.random.shuffle(batch)
                yield batch
                batch = []
        if len(batch) > 0 and not self.drop_last:
            yield batch

    def __len__(self):
        if self.drop_last:
            return len(self.total_index) // self.batch_size
        else:
            return (len(self.total_index) + self.batch_size - 1) // self.batch_size
        

from torch.utils.data import Dataset

class PlaceholderDataset(Dataset):
    def __init__(self, data_source):
        self.data_source = data_source

    def __len__(self):
        return len(self.data_source)

    def __getitem__(self, index):
        raise NotImplementedError("This should not be called because the sampler handles data loading.")

import torch
from torch.utils.data import DataLoader

# 假设 pos_data 和 neg_data 已经是加载到内存的列表或张量
pos_data = [torch.tensor(data) for data in range(100)]
neg_data = [torch.tensor(data) for data in range(100, 200)]

# 索引通常是连续的整数列表
pos_index = list(range(100))
neg_index = list(range(100))
total_index = list(range(100))  # 假设我们只需要50个批次

# 实例化 CustomBatchSampler
sampler = CustomBatchSampler(pos_data, neg_data, pos_index, neg_index, total_index, batch_size=4, drop_last=True)




100, 100, 100, 0


NotImplementedError: This should not be called because the sampler handles data loading.

In [26]:

from torch.utils.data import Dataset
import os

class CustomDataset(Dataset):
    def __init__(self, data_dir, data_length, offset, pos):
        self.data_dir = data_dir
        self.split_files = [f for f in os.listdir(data_dir) if f.startswith("data_")]
        self.cumulative_lengths = np.cumsum([self.get_length(f) for f in self.split_files])
        self.pos = pos
        self.length = data_length
        self.offset = offset
        self.data_file_idx = None
        self.data = None

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        idx = (idx + self.offset)%self.cumulative_lengths[-1]
        split_idx = np.searchsorted(self.cumulative_lengths, idx + 1)
        if split_idx == 0:
            sample_idx = idx
        else:
            sample_idx = idx - self.cumulative_lengths[split_idx - 1]

        return self.load_sample(self.split_files[split_idx], sample_idx)

    def get_length(self, split_file):
        data = np.load(os.path.join(self.data_dir, split_file)) 
        return len(data)

    def load_sample(self, split_file, idx):
        if self.data_file_idx is None or self.data_file_idx != split_file:
            self.data = np.load(os.path.join(self.data_dir, split_file))
            self.data_file_idx = split_file
        sample = self.data[idx]
        # Preprocess the data if needed
        sample = torch.tensor(sample, dtype=torch.float32)  # Convert to PyTorch tensor if needed
        if self.pos:
            target = torch.tensor(1, dtype=torch.int64)
        else:
            target = torch.tensor(0, dtype=torch.int64)
        return sample, target


# # 测试custom数据集
# from torch.utils.data import DataLoader
# data_length = 6000000
# neg_dataset = CustomDataset("../../data/features_0507/neg", data_length=data_length, offset=1000000, pos=False)
# neg_loader = DataLoader(neg_dataset, batch_size=2, shuffle=True)
# pos_dataset = CustomDataset("../../data/features_0507/pos", data_length=data_length, offset=1000000, pos=True)
# pos_loader = DataLoader(pos_dataset, batch_size=2, shuffle=True)



In [42]:
for i, (neg, pos) in enumerate(zip(neg_loader, pos_loader)):
    print(pos)
    if i > 10:
        break

[tensor([[ 0.0051,  0.0026,  0.0269,  ..., -0.0077, -0.0232,  0.0515],
        [ 0.0347,  0.0660,  0.0250,  ..., -0.0105, -0.0411,  0.0453]]), tensor([1, 1], dtype=torch.int32)]
[tensor([[ 0.0356,  0.0104,  0.0217,  ...,  0.0211, -0.0627,  0.0385],
        [ 0.0591,  0.0131,  0.0225,  ...,  0.0122, -0.0498,  0.0432]]), tensor([1, 1], dtype=torch.int32)]
[tensor([[-0.0269, -0.0255,  0.0282,  ...,  0.0079, -0.0407,  0.0515],
        [ 0.0226,  0.0339,  0.0221,  ...,  0.0181, -0.0486,  0.0553]]), tensor([1, 1], dtype=torch.int32)]
[tensor([[-0.0011,  0.0155,  0.0161,  ...,  0.0207, -0.0476,  0.0344],
        [ 0.0015,  0.0052,  0.0220,  ...,  0.0225, -0.0266,  0.0260]]), tensor([1, 1], dtype=torch.int32)]
[tensor([[-0.0010,  0.0063,  0.0220,  ...,  0.0385, -0.0262,  0.0468],
        [-0.0056,  0.0706,  0.0092,  ..., -0.0102, -0.0434,  0.0278]]), tensor([1, 1], dtype=torch.int32)]
[tensor([[-0.0268, -0.0103, -0.0044,  ...,  0.0182, -0.0195,  0.0037],
        [-0.0273,  0.0533,  0.0219,  ..

In [10]:
from torch.autograd import Variable
import torch.nn.functional as F

def mixup_cross_entropy_loss(input, target, size_average=True):
    """Origin: https://github.com/moskomule/mixup.pytorch
    in PyTorch's cross entropy, targets are expected to be labels
    so to predict probabilities this loss is needed
    suppose q is the target and p is the input
    loss(p, q) = -\sum_i q_i \log p_i
    """
    assert input.size() == target.size()
    assert isinstance(input, Variable) and isinstance(target, Variable)
    input = torch.log(torch.nn.functional.softmax(input, dim=1).clamp(1e-5, 1))
    # input = input - torch.log(torch.sum(torch.exp(input), dim=1)).view(-1, 1)
    loss = - torch.sum(input * target)
    return loss / input.size()[0] if size_average else loss


class FocalLoss(nn.Module):
    def __init__(self, class_num, alpha=0.20, gamma=1.5, use_alpha=False, size_average=True, device='cpu'):
        super(FocalLoss, self).__init__()
        self.class_num = class_num
        self.alpha = alpha
        self.gamma = gamma
        self.device = device
        if use_alpha:
            self.alpha = torch.tensor(alpha).to(device)
            # self.alpha = torch.tensor(alpha)

        self.softmax = nn.Softmax(dim=1)
        self.use_alpha = use_alpha
        self.size_average = size_average

    def forward(self, pred, target):

        prob = self.softmax(pred.view(-1,self.class_num))
        prob = prob.clamp(min=0.0001,max=1.0)

        target_ = torch.zeros(target.size(0),self.class_num).to(self.device)
        # target_ = torch.zeros(target.size(0),self.class_num)
        target_.scatter_(1, target.view(-1, 1).long(), 1.)

        if self.use_alpha:
            batch_loss = - self.alpha.double() * torch.pow(1-prob,self.gamma).double() * prob.log().double() * target_.double()
        else:
            batch_loss = - torch.pow(1-prob,self.gamma).double() * prob.log().double() * target_.double()

        batch_loss = batch_loss.sum(dim=1)

        if self.size_average:
            loss = batch_loss.mean()
        else:
            loss = batch_loss.sum()

        return loss

In [25]:
def get_dataloader_by_fold_index_ver(pos_dir, neg_dir, total_len, fold_num, folds, train_batch_size=1024, val_batch_size=32, collate_fn=None):
    """
    通过交叉验证的fold数获取训练和验证数据集的DataLoader。
    
    参数:
    X (np.array): 特征数据
    y (np.array): 标签数据
    fold_num (int): 交叉验证的fold数
    folds (int): 总的fold数
    
    返回:
    DataLoader, DataLoader: 训练和验证数据集的DataLoader
    """
    
    # 计算每个fold的大小
    fold_size = total_len // folds
    # 计算验证集的起始和大小
    val_start = fold_size * fold_num
    val_size = fold_size
    # 选择验证集
    pos_val = CustomDataset(pos_dir, val_size, val_start, pos=True)
    neg_val = CustomDataset(neg_dir, val_size, val_start, pos=False)
    # 选择训练集
    pos_train = CustomDataset(pos_dir, total_len-val_size, val_start+val_size, pos=True)
    neg_train = CustomDataset(neg_dir, total_len-val_size, val_start+val_size, pos=False)
    
    # 创建数据集和DataLoader
    if collate_fn is None:
        pos_train_loader = DataLoader(pos_train, batch_size=train_batch_size, shuffle=True)
        neg_train_loader = DataLoader(neg_train, batch_size=train_batch_size, shuffle=True)
    else:
        pos_train_loader = DataLoader(pos_train, batch_size=train_batch_size, shuffle=True, collate_fn=collate_fn)
        neg_train_loader = DataLoader(neg_train, batch_size=train_batch_size, shuffle=True, collate_fn=collate_fn)
    pos_val_loader = DataLoader(pos_val, batch_size=val_batch_size, shuffle=False)
    neg_val_loader = DataLoader(neg_val, batch_size=val_batch_size, shuffle=False)

    return pos_train_loader, neg_train_loader, pos_val_loader, neg_val_loader

In [ ]:
def get_dataloader_by_fold(X, y, fold_num, folds, train_batch_size=1024, val_batch_size=32, collate_fn=None):
    """
    通过交叉验证的fold数获取训练和验证数据集的DataLoader。
    
    参数:
    X (np.array): 特征数据
    y (np.array): 标签数据
    fold_num (int): 交叉验证的fold数
    folds (int): 总的fold数
    
    返回:
    DataLoader, DataLoader: 训练和验证数据集的DataLoader
    """
    # 计算每个fold的大小
    fold_size = len(X) // folds
    # 计算验证集的起始和结束位置
    val_start = fold_size * fold_num
    val_end = fold_size * (fold_num + 1)
    # 选择验证集
    X_val = X[val_start:val_end]
    y_val = y[val_start:val_end]
    dataset_val = Embed_data(X_val, y_val)
    # 选择训练集
    X_train = np.concatenate([X[:val_start], X[val_end:]])
    y_train = np.concatenate([y[:val_start], y[val_end:]])
    dataset_train = Embed_data(X_train, y_train)

    # 创建数据集和DataLoader
    if collate_fn is None:
        train_loader = DataLoader(dataset_train, batch_size=train_batch_size, shuffle=False)
    else:
        train_loader = DataLoader(dataset_train, batch_size=train_batch_size, shuffle=False, collate_fn=collate_fn)
    val_loader = DataLoader(dataset_val, batch_size=val_batch_size, shuffle=False)
    
    return train_loader, val_loader

In [21]:
total_index, X_pos_index, X_neg_index = balance_data_by_number(len(X_pos), 6000000)
pos_dataset = CustomDataset("../../data/features_0507/pos", pos=True)
neg_dataset = CustomDataset("../../data/features_0507/neg", pos=False)
# X_balanced_index, X_all, y_all = balance_data(X_pos, X_neg)
# X_balanced_test, y_balanced_test = balance_data(X_pos_test, y_pos_test, X_neg_test, y_neg_test)

pos = (6000000,), neg = (6000000,)


In [30]:
mixup = RandomMixUp(2,p=0.8,alpha=1.5)
def collate_fn(batch):
    return mixup(*default_collate(batch))

# train_loader, test_loder, y_test = get_dataloader_by_fold(X_balanced, y_balanced, 0, 5, collate_fn=collate_fn)

mlp = MLP(768, [1024, 2048, 1024, 512, 256, 128], 2)
model_name = "mlp_new_total_data"
# loss_function = nn.CrossEntropyLoss(label_smoothing=0.)
loss_function = FocalLoss(2, gamma=1.5)
optimizer = torch.optim.AdamW(  
    mlp.parameters(), 
    lr=0.01,
    weight_decay=0.02)


lr_scheduler = CosineAnnealingLR(
    optimizer,
    max_epochs=20,
    warmup_epochs=8,
    warmup_start_lr=0.001,
    eta_min=0.00001)

In [31]:
best_acc = 0
folds = 10

for epoch in range(1,11):
    print(f'Epoch {epoch}')
    current_loss = 0
    pos_train_loader, neg_train_loader, pos_val_loader, neg_val_loader = get_dataloader_by_fold_index_ver("../../data/features_0507/pos", "../../data/features_0507/neg", 6000000, epoch%folds, folds, collate_fn=collate_fn)
    mlp.train()
    for i, (pos, neg) in enumerate(zip(pos_train_loader, neg_train_loader)):
        X = torch.cat((pos[0], neg[0]), dim=0)
        y = torch.cat((pos[1], neg[1]), dim=0)
        # print(X.shape)
        optimizer.zero_grad()
        y_pred = mlp(X.float())
        # y_pred = torch.nn.functional.one_hot(y_pred.argmax(dim=1), num_classes=2).to(dtype=y_pred.dtype)
        # print(y_pred.size(), y.size())
        y = np.argmax(y, axis=1)
        # print(y.size())
        loss = loss_function(y_pred.squeeze(), y.float())
        loss.backward()
        optimizer.step()
        current_loss += loss.item()
        if i % 100 == 0:
            print(f'Epoch {epoch} Loss: {current_loss/(i+1)}')
    print(f"TOTAL EPOCH {epoch} LOSS: {current_loss/(len(pos_train_loader)+len(neg_train_loader))}")

    mlp.eval()
    y_pred = []
    y_test = []
    with torch.no_grad():
        for i, (pos, neg) in enumerate(zip(pos_val_loader, neg_val_loader)):
            X = torch.cat((pos[0], neg[0]), dim=0)
            y = torch.cat((pos[1], neg[1]), dim=0).cpu().numpy()
            res = mlp(X.float())
            res = torch.nn.functional.softmax(res, dim=1)
            res = res.cpu().numpy()
            res = np.argmax(res, axis=1)
            # print(res.shape)
            y_pred.append(res)
            y_test.append(y)
    y_pred = np.hstack(y_pred)
    y_test = np.hstack(y_test)
    acc = ((y_pred >= 0.5) == y_test).mean()
    if acc > best_acc:
        best_acc = acc
        torch.save(mlp.state_dict(), f'model/{model_name}_best.pth')
        print(f'Model saved')
    print(f'acc={acc}')
    
    
print(f'Training finished! {best_acc = }')

Epoch 1
Epoch 1 Loss: 0.24524827103339894
Epoch 1 Loss: 0.032181247937702995
Epoch 1 Loss: 0.02400422465005205
Epoch 1 Loss: 0.020940729093439626
Epoch 1 Loss: 0.019199602888258047
Epoch 1 Loss: 0.01787778876438195
Epoch 1 Loss: 0.017205090559298242
Epoch 1 Loss: 0.016654252539682637
Epoch 1 Loss: 0.016180604483238
Epoch 1 Loss: 0.015845411991698154
Epoch 1 Loss: 0.015443060335041347
Epoch 1 Loss: 0.015262936142695572
Epoch 1 Loss: 0.015133416635131479
Epoch 1 Loss: 0.014911587907258682
Epoch 1 Loss: 0.014770836155850346
Epoch 1 Loss: 0.014594568633051797
Epoch 1 Loss: 0.014479817595155566
Epoch 1 Loss: 0.01432326307074204
Epoch 1 Loss: 0.014223586222831663
Epoch 1 Loss: 0.014088129876624764
Epoch 1 Loss: 0.013969920993082249
Epoch 1 Loss: 0.01391563650392678
Epoch 1 Loss: 0.013843014094254343
Epoch 1 Loss: 0.013747330827308127
Epoch 1 Loss: 0.013636976469769125
Epoch 1 Loss: 0.013592357205411732
Epoch 1 Loss: 0.013580147168591825
Epoch 1 Loss: 0.013553955059235356
Epoch 1 Loss: 0.0135

In [23]:
y.shape


torch.Size([1024, 2])

In [133]:
mlp.eval()
y_pred = []
with torch.no_grad():
    for X, y in test_loader:
        res = mlp(X.float())
        res = torch.nn.functional.softmax(res, dim=1)
        res = res.cpu().numpy()
        res = np.argmax(res, axis=1)
        # print(res.shape)
        y_pred.append(res)
y_pred = np.hstack(y_pred)
print(f'MSE: {mean_squared_error(y_balanced_test, y_pred)}')
print(f'Accuracy: {((y_pred >= 0.5) == y_balanced_test).mean()}')

MSE: 0.03055773682913131
Accuracy: 0.9694422631708687


In [121]:
loss_function

FocalLoss(
  (softmax): Softmax(dim=1)
)

In [69]:
# 保存模型
torch.save(mlp.state_dict(), 'model/mlp_image_embed.pth')
print('Model saved')

Model saved


In [34]:
# 加载模型
mlp = MLP(768, [256, 128, 64], 1)
mlp.load_state_dict(torch.load('model/mlp_best.pth'))
mlp.eval()
print('Model loaded')

Model loaded


In [8]:
label_file = '../labels.txt'
label_file = '../../data/data/Person/label/filter_train.txt'
with open(label_file, 'r') as f:
    labels = f.readlines()
    labels = [label.strip() for label in labels]

for i in range(10):
    print(labels[i])

Person_Det/data/train/XWKJ-1010100001_83910152-4cc6-11ec-93bf-0242ac110004_240933_origin.jpg 1945 1117 1987 1222 0
Person_Det/data/train/XWKJ-1010100001_9af894a8-4ab0-11ec-9ead-0242ac110002_211751_origin.jpg 2383 654 2426 761 0
Person_Det/data/train/XWKJ-1010100001_a10873f6-48f1-11ec-be53-0242ac110004_191232_origin.jpg 2326 1106 2384 1251 0
Person_Det/data/train/XWKJ-1010100001_36d558a8-4c04-11ec-88bc-0242ac110004_231023_origin.jpg 2486 1103 2534 1223 0
Person_Det/data/train/XWKJ-1010100001_7995147c-49d8-11ec-9bd8-0242ac110004_201605_origin.jpg 306 94 363 236 0
Person_Det/data/train/XWKJ-1010100001_6ae612b2-4697-11ec-b0c6-0242ac110003_161241_origin.jpg 542 674 588 789 0
Person_Det/data/train/XWKJ-1010100001_315a4b78-50bd-11ec-8090-0242ac110002_291036_origin.jpg 1574 708 1628 843 0
Person_Det/data/train/XWKJ-1010100001_7741aeee-41bb-11ec-9965-0242ac110004_100817_origin.jpg 1054 288 1109 425 0
Person_Det/data/train/XWKJ-1010100001_02b6352e-4a6f-11ec-aad7-0242ac110002_211001_origin.jpg 17

In [7]:
def balanced_sampler(X_pos, y_pos, X_neg, y_neg, batch_size):
    num_pos = len(X_pos)
    num_neg = len(X_neg)
    total_samples = max(num_pos, num_neg)
    

    # Shuffle positive and negative samples
    pos_indices = np.random.permutation(num_pos)
    neg_indices = np.random.permutation(num_neg)
    
    # Generate balanced batches
    for i in range(0, total_samples, batch_size):
        # Repeat sampling if one class is exhausted
        pos_batch_indices = pos_indices[i:i+batch_size]
        neg_batch_indices = neg_indices[i:i+batch_size]
        if len(pos_batch_indices) < batch_size:
            pos_batch_indices = np.random.choice(pos_indices, size=batch_size, replace=True)
        if len(neg_batch_indices) < batch_size:
            neg_batch_indices = np.random.choice(neg_indices, size=batch_size, replace=True)
            
        # Yield balanced batch samples and labels
        yield np.concatenate([X_pos[pos_batch_indices], X_neg[neg_batch_indices]]), \
              np.concatenate([y_pos[pos_batch_indices], y_neg[neg_batch_indices]])

In [9]:
# clf = SGDClassifier(alpha=.0001, loss='hinge', penalty='l2', n_jobs=-1, shuffle=True, max_iter=10000, verbose=0, tol=0.001)
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(512), random_state=1)

sampler = balanced_sampler(X_pos_train, y_pos_train, X_neg_train, y_neg_train, 1024)

acc_best = 0

n_epoch = 5
for _ in range(n_epoch):
    for index, (X_batch, y_batch) in enumerate(sampler):
        clf.partial_fit(X_batch, y_batch, classes=[0, 1])

        y_pos_predicted = clf.predict(X_pos_test)
        acc_pos = accuracy_score(y_pos_test, y_pos_predicted)

        y_neg_predicted = clf.predict(X_neg_test)
        acc_neg = accuracy_score(y_neg_test, y_neg_predicted)

        acc_avg = (acc_pos + acc_neg) / 2.

        if acc_best < acc_avg:
            acc_best = acc_avg
            joblib.dump(clf, 'result/person_classifier.pkl')

        print(f'avg={acc_avg}, pos={acc_pos}, neg={acc_neg}')

print(f'acc_best={acc_best}')

AttributeError: partial_fit is only available for stochastic optimizer. lbfgs is not stochastic